# Statistical significance of GW170814 signal
GW170814 event was observed by all interferometers.

## 1. Prepare the data.

<p>Download datastream from the public catalog.</p>

<p> Filter out the low-frequency contribution and remove the high-frequency. Instead of using a filter remove high-frequency components by resampling the timeseries. If needed check excercise "17-Template fit and q-transform" and follow data conditioning part.</p>

In [ ]:
%matplotlib inline

import pylab
from pycbc.filter import resample_to_delta_t, highpass
from pycbc.catalog import Merger
from pycbc.psd import interpolate, inverse_spectrum_truncation

m = Merger("GW170814")

ifos = ['H1', 'L1', 'V1']
data = {}  # We need to "alocate" variable name to use in the for loop
psd = {}

pylab.figure(figsize=[10, 5])

for ifo in ifos:
    # Read in and precondition the data
    ts = m.strain(ifo).highpass_fir(15, 512)
    data[ifo] = resample_to_delta_t(ts, 1.0/2048).crop(2, 2)  # Low-pass and crop

    # Estimate the power spectral density of the data
    # This chooses to use 4s samples in the PSD estimate.
    # One should note that the tradeoff in segment length is that
    # resolving narrow lines becomes more difficult.
    p = data[ifo].psd(4)
    p = interpolate(p, data[ifo].delta_f)
    p = inverse_spectrum_truncation(p, int(4 * data[ifo].sample_rate), low_frequency_cutoff=20.0)
    psd[ifo] = p
    
    pylab.plot(psd[ifo].sample_frequencies, psd[ifo], label=ifo)

pylab.yscale('log')
pylab.xscale('log')
pylab.ylim(1e-47, 1e-41)
pylab.xlim(20, 1024)
pylab.ylabel('$Strain^2 / Hz$')
pylab.xlabel('Frequency (Hz)')
pylab.grid()
pylab.legend()
pylab.show()

## 2. Generate the template and calculate the SNR.

In [ ]:
from pycbc.waveform import get_fd_waveform
from pycbc.filter import matched_filter
from pycbc.conversions import mass1_from_mchirp_q

# Calculate the component mass of each black hole in the detector frame
# http://pycbc.org/pycbc/latest/html/_modules/pycbc/catalog.html#Merger.median1d
mchirp= m.median1d("mchirp") # This is in the source frame
zfac = (1 + m.median1d("redshift")) # apply redshift to get to the detector frame
mchirp *= zfac
mass1 = mass2 = mass1_from_mchirp_q(mchirp, 1)

# This is a frequency domain waveform generator.
# Compare with get_td_waveform() in excercise "17-Template fit and q-transform"
# This function returns both a plus and a cross
# polarization waveform, but we will just use the plus polarization in building our template
# as these are only different by a phase offset in this specific case.
hp, _ = get_fd_waveform(approximant="IMRPhenomD",
                        mass1=mass1, mass2=mass2,
                        f_lower=20.0, delta_f=data[ifo].delta_f)
hp.resize(len(psd[ifo]))

# For each observatory use this template to calculate the SNR time series
snr = {}
for ifo in ifos:
    snr[ifo] = matched_filter(hp, data[ifo], psd=psd[ifo], low_frequency_cutoff=25)
    snr[ifo] = snr[ifo].crop(5, 4)

Check median1d function

## Plot all data and zoom around the event

In [ ]:
# Show a couple sizes
for w, title in [(7, 'Wide View'), (.15, 'Close to GW170814')]:
    pylab.figure(figsize=[14, 4])
    for ifo in ifos:
        pylab.plot(snr[ifo].sample_times, abs(snr[ifo]), label=ifo)

    pylab.legend()
    pylab.title(title)
    pylab.grid()
    pylab.xlim(m.time - w, m.time + w)
    pylab.ylim(0, 15)
    pylab.xlabel('Time (s)')
    pylab.ylabel('Signal-to-noise (SNR)')
    pylab.show()

## 3. Calculate peak significance.

Use $\chi^2$ method and calculate the significance for signal from all observatories.

In [ ]:
from pycbc.vetoes import power_chisq
chisq = {}
for ifo in ifos:
    # The number of bins to use. In principle, this choice is arbitrary. In practice,
    # this is empirically tuned.
    nbins = 26
    chisq[ifo] = power_chisq(hp, data[ifo], nbins, psd[ifo], low_frequency_cutoff=20.0)
    chisq[ifo] = chisq[ifo].crop(5, 4)
    
    dof = nbins * 2 - 2
    chisq[ifo] /= dof

Plot the normalized $\chi^2$.

In [ ]:
pylab.figure(figsize=[14, 4])

for ifo in ifos:
    pylab.plot(chisq[ifo].sample_times, chisq[ifo], label=ifo)
    
pylab.legend()
pylab.grid()
pylab.xlim(m.time -0.15, m.time + 0.15)
pylab.ylim(0, 5)
pylab.ylabel('$chi^2_r$')
pylab.show()